## News :D



In [15]:
%load_ext autoreload
%autoreload 2

import requests
import re
import itertools
from bs4 import BeautifulSoup

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Installs:
lxmn
bs4
pytorch

In [ ]:
months = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]
mday = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# CNN
cnn_dict = dict(
    [
        (
            i,
            [
                f"https://www.cnn.com/article/sitemap-{i}-{j}.html"
                for j in range(1, 13)
            ],
        )
        for i in range(2014, 2025)
    ]
)


# New York Times
nyt_dict = dict(
    [
        (
            i,
            dict(
                [
                    (
                        j,
                        [
                            f"https://www.nytimes.com/sitemap/{i}/{str(j).zfill(2)}/{str(k).zfill(2)}/"
                            for k in range(1, mday[j - 1]+1)
                        ],
                    )
                    for j in range(1, 13)
                ]
            ),
        )
        for i in range(2014, 2025)
    ]
)


# Business Insider
bi_dict = dict(
    [
        (
            i,
            [
                f"https://www.businessinsider.com/sitemap/html/{i}-{str(j).zfill(2)}.html"
                for j in range(1, 13)
            ],
        )
        for i in range(2014, 2025)
    ]
)


# The Hill
th_dict = dict(
    [
        (
            i,
            dict(
                [
                    (
                        months.index(j)+1,
                        [
                            f"https://thehill.com/sitemap/{i}/{j}/{k}/"
                            for k in range(1, mday[months.index(j) - 1]+1)
                        ],
                    )
                    for j in months
                ]
            ),
        )
        for i in range(2014, 2025)
    ]
)


# New York Post
nyp_dict = dict(
    [
        (
            i,
            dict(
                [
                    (
                        j,
                        [
                            f"https://nypost.com/{i}/{str(j).zfill(2)}/{str(k).zfill(2)}/"
                            for k in range(1, mday[j - 1]+1)
                        ],
                    )
                    for j in range(1, 13)
                ]
            ),
        )
        for i in range(2014, 2025)
    ]
)


# Daily Mail
dp_dict = dict(
    [
        (
            i,
            dict(
                [
                    (
                        j,
                        [
                            f"https://www.dailymail.co.uk/home/sitemaparchive/month_{i}{str(j).zfill(2)}{str(k).zfill(2)}.html"
                            for k in range(1, mday[j - 1]+1)
                        ],
                    )
                    for j in range(1, 13)
                ]
            ),
        )
        for i in range(2014, 2025)
    ]
)


# Fox
fox_links = ["https://www.foxnews.com/sitemap.xml?type=articles"] + [
    f"https://www.foxnews.com/sitemap.xml?type=articles&page={i}"
    for i in range(2, 161)
]

{1: ['https://www.dailymail.co.uk/home/sitemaparchive/month_20240101.html', 'https://www.dailymail.co.uk/home/sitemaparchive/month_20240102.html', 'https://www.dailymail.co.uk/home/sitemaparchive/month_20240103.html', 'https://www.dailymail.co.uk/home/sitemaparchive/month_20240104.html', 'https://www.dailymail.co.uk/home/sitemaparchive/month_20240105.html', 'https://www.dailymail.co.uk/home/sitemaparchive/month_20240106.html', 'https://www.dailymail.co.uk/home/sitemaparchive/month_20240107.html', 'https://www.dailymail.co.uk/home/sitemaparchive/month_20240108.html', 'https://www.dailymail.co.uk/home/sitemaparchive/month_20240109.html', 'https://www.dailymail.co.uk/home/sitemaparchive/month_20240110.html', 'https://www.dailymail.co.uk/home/sitemaparchive/month_20240111.html', 'https://www.dailymail.co.uk/home/sitemaparchive/month_20240112.html', 'https://www.dailymail.co.uk/home/sitemaparchive/month_20240113.html', 'https://www.dailymail.co.uk/home/sitemaparchive/month_20240114.html', '

In [34]:
r = requests.get(cnn_dict[2024][0])
soup = BeautifulSoup(r.text, "html.parser")
test = [i for i in soup.find_all("li")]
titles = []
links = []
dates = []
for i in test:
    for j in i.descendants:
        if "sitemap-link" in str(j):
            titles.append(
                str(j)[
                    36
                    + str(j)[36:].index('"')
                    + 2 : 36
                    + str(j)[36:].index("<")
                ]
            )
            links.append(str(j)[36 : 36 + str(j)[36:].index('"')])
        if "date" in str(j):
            dates.append(str(j)[19:29])

print(titles)
print(links)
print(dates)

r = requests.get(nyt_dict[2024][1][0])
soup = BeautifulSoup(r.text, "html.parser")
# print(r.text)
test = [i for i in soup.find_all("li")]
titles = []
links = []
dates = []
for i in test:
    test = str(i.contents[0])
    if ("https:" in test) & (".html" in test) & ("2024/01/01" in test):
        links.append(test[test.index("https:") : test.index(".html")])
        titles.append(test[test.index(">") + 1 : test[4:].index("<") + 4])
        dates.append("2024-01-01")

print(titles)
print(links)
print(dates)




# <span class="sitemap-link">
# <li>
# <span class="date">2024-01-31</span><span class="sitemap-link">
# <a href="https://www.cnn.com/tech/live-news/meta-x-discord-tiktok-snap-chiefs-testimony-senate/index.html">
# Mark Zuckerberg apologizes to families over social media harms in contentious Senate hearing
# </a>
# </span>
# </li>

["Today's news in 10 minutes", 'Mark Zuckerberg apologizes to families over social media harms in contentious Senate hearing', "Stocks sink as Fed hints rate cuts aren't coming soon", 'Simp: The slang teenagers use to insult boys', "New Hampshire's 2024 primary will be a crucial early test of Trump's comeback bid", "'I think he's the best guy for the job': See the 2024 campaign through the eyes of Iowa voters", 'Abortion divides Iowa GOP voters ahead of crucial first primary debate', 'Some Iowa voters shift favorites after GOP debate, while Ramaswamy stokes a divide', 'Bill Frist Fast Facts', 'Australian Open Fast Facts', 'John McEnroe Fast Facts', 'Death Penalty Fast Facts', "Opinion: India's 'murder most foul' has a chilling subtext", 'January 30, 2024 Israel-Hamas war', "Today's news in 10 minutes", '2024 Presidential Debates Fast Facts', 'What is catfishing and what can you do if you are catfished?', 'January 29 2024 Israel-Hamas war', "Judge denies Murdaugh's motion for new trial"

In [ ]:
# Pandas Testing area
import pandas as pd

test_frame = pd.DataFrame()